In [233]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd 
import numpy as np
import seaborn as sns
from re import sub
import json
import requests
import time
from bs4 import BeautifulSoup as bs

sns.set_style("darkgrid")

In [234]:
%%capture
from tqdm import tqdm_notebook as tqdm

In [235]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

# Read CSVs

In [236]:
# mov_df1 = pd.read_csv("data/en_csv.csv")
mov12 = pd.read_csv("data/five_year.csv")

scrape = pd.read_csv("data/scrape.csv")


## Concatenate DataFrames

In [237]:
# mov12 = pd.concat([mov_df1, mov_df2], sort=False)

In [238]:
mov12.shape

(49978, 30)

In [239]:
scrape.drop(['Unnamed: 0'], axis = 1, inplace=True)
scrape.drop_duplicates(inplace=True)
mov12.drop_duplicates(inplace=True)

In [240]:
dfm = mov12.merge(scrape, how='left', on="imdb_id")

In [241]:
dfm.loc[dfm.budget_x == 0,'budget_x'] = dfm.loc[dfm.budget_x == 0,'budget_y']
dfm.loc[dfm.revenue_x == 0,'revenue_x'] = dfm.loc[dfm.revenue_x == 0,'revenue_y']
dfm.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'adult', 'backdrop_path',
       'belongs_to_collection', 'budget_x', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue_x', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'success', 'status_code',
       'status_message', 'budget_y', 'revenue_y'],
      dtype='object')

In [242]:
cols =['Unnamed: 0', 'Unnamed: 0.1', 'adult', 'backdrop_path',
       'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'success', 'status_code',
       'status_message', 'budget_y', 'revenue_y']
dfm.columns = cols

In [243]:
dfm.dropna(subset=['genres', 'budget', 'revenue', 'vote_average'], inplace=True)
dfm.shape

(2309, 32)

In [244]:
dfm.describe()

,Unnamed: 0,Unnamed: 0.1,budget,id,popularity,revenue,runtime,vote_average,vote_count,status_code,budget_y,revenue_y
count,2309.000000,2309.000000,2.309000e+03,2309.000000,2309.000000,2.309000e+03,2276.000000,2309.000000,2309.000000,0.0,1.164000e+03,1.164000e+03
mean,2350.071460,2350.071460,9.285317e+07,459739.336942,20.748039,5.453184e+07,101.745167,5.907492,874.903421,NaN,1.536521e+08,4.997352e+06
std,2544.761509,2544.761509,1.156119e+09,110233.590836,50.718568,1.738905e+08,32.718514,2.084265,2212.912168,NaN,1.625621e+09,2.598041e+07
min,0.000000,0.000000,0.000000e+00,14564.000000,0.600000,1.000000e+00,0.000000,0.000000,0.000000,NaN,0.000000e+00,3.700000e+01
25%,280.000000,280.000000,1.000000e+06,391995.000000,2.057000,6.239800e+04,90.000000,5.500000,5.000000,NaN,7.406250e+05,2.361725e+04
50%,1293.000000,1293.000000,5.000000e+06,450045.000000,6.504000,1.598584e+06,102.000000,6.400000,40.000000,NaN,2.850000e+06,1.827230e+05
75%,3743.000000,3743.000000,2.498700e+07,525176.000000,19.619000,2.267856e+07,120.000000,7.000000,523.000000,NaN,1.012500e+07,1.490424e+06
max,9972.000000,9972.000000,3.500000e+10,761122.000000,1114.882000,2.797801e+09,227.000000,10.000000,23833.000000,NaN,3.500000e+10,4.511834e+08


In [245]:
dfm.head()

,Unnamed: 0,Unnamed: 0.1,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,...,tagline,title,video,vote_average,vote_count,success,status_code,status_message,budget_y,revenue_y
0,0,0,False,/ld7V9BjMk2xtiBNcR8savyyk5ca.jpg,NaN,110000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",http://www.foxmovies.com/movies/miss-peregrine...,283366.0,tt1935859,...,Stay peculiar,Miss Peregrine's Home for Peculiar Children,False,6.7,7533.0,NaN,NaN,NaN,NaN,NaN
2,2,2,False,/craD86vySKvAkboyeXFnZwHrNA8.jpg,"{'id': 256322, 'name': 'The Purge Collection',...",10000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",http://www.purgeelectionyear.com,316727.0,tt4094724,...,Keep America great,The Purge: Election Year,False,6.4,3735.0,NaN,NaN,NaN,NaN,NaN
3,3,3,False,/mMtUybQ6hL24FXo0F3Z4j2KG7kZ.jpg,NaN,370000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 16, ...",https://www.funimationfilms.com/movie/yourname/,372058.0,tt5311514,...,NaN,Your Name.,False,8.6,6597.0,NaN,NaN,NaN,370000000.0,358922706.0
4,4,4,False,/z9ft5HYHzWcasR6SGcgeluxTznB.jpg,"{'id': 544670, 'name': 'Sing Collection', 'pos...",75000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://www.singmovie.com/,335797.0,tt3470600,...,Auditions begin 2016.,Sing,False,7.0,5631.0,NaN,NaN,NaN,NaN,NaN
5,5,5,False,/6O0lsCK90jZCyPvyYSt8Szzlnd6.jpg,NaN,40000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",https://www.lionsgate.com/movies/hacksaw-ridge,324786.0,tt2119532,...,One of the greatest heroes in American history...,Hacksaw Ridge,False,8.1,8829.0,NaN,NaN,NaN,NaN,NaN


In [246]:
dfm = dfm.loc[dfm['budget'] > 50000, :]
dfm = dfm.loc[dfm['revenue'] > 50000, :]
dfm = dfm.loc[dfm['vote_average'] > .01, :]

In [247]:
dfm.shape

(1657, 32)

In [248]:
dfm['Unnamed: 0'].value_counts()

115     5
82      5
56      5
49      5
13      5
       ..
1114    1
1112    1
1109    1
246     1
778     1
Name: Unnamed: 0, Length: 1192, dtype: int64

In [249]:
cols = ['imdb_id', 'title', 'genres', 'popularity', 'release_date', 'budget', 'revenue', 'vote_average' ]
cols_to_drop = [x for x in mov_df2.columns if x not in cols]

In [250]:
df = dfm.copy()
df.drop(columns = cols_to_drop, inplace=True)
df = df.reindex(columns=cols)
df

,imdb_id,title,genres,popularity,release_date,budget,revenue,vote_average
0,tt1935859,Miss Peregrine's Home for Peculiar Children,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",306.289,2016-09-28,110000000.0,296482446.0,6.7
2,tt4094724,The Purge: Election Year,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",131.741,2016-06-29,10000000.0,118587880.0,6.4
3,tt5311514,Your Name.,"[{'id': 10749, 'name': 'Romance'}, {'id': 16, ...",130.125,2016-08-26,370000000.0,357986087.0,8.6
4,tt3470600,Sing,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",96.344,2016-11-23,75000000.0,634151679.0,7.0
5,tt2119532,Hacksaw Ridge,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",104.716,2016-10-07,40000000.0,175302354.0,8.1
...,...,...,...,...,...,...,...,...
45857,tt9199374,Uriyadi,[],0.840,2020-01-17,3997866.0,11327287.0,5.0
47199,tt10157810,Fools in the Mountains,"[{'id': 35, 'name': 'Comedy'}]",0.791,2020-02-21,21000000.0,1020756.0,5.5
47509,tt9861288,Boutchou,"[{'id': 35, 'name': 'Comedy'}]",0.846,2020-09-23,4200000.0,1227428.0,5.5
47911,NaN,ALL COPS AIN'T BAD,"[{'id': 35, 'name': 'Comedy'}]",1.230,2020-01-01,100000.0,350000.0,10.0


In [251]:
df['ROI'] = df['revenue'] / df['budget']

In [252]:
df.release_date.sort_values()

260      2016-01-01
542      2016-01-01
2236     2016-01-06
234      2016-01-07
235      2016-01-07
            ...    
40870    2020-09-25
40026    2020-10-02
43207    2020-10-14
40544    2020-10-16
41098    2020-10-21
Name: release_date, Length: 1657, dtype: object

In [253]:
df.to_csv("data/cleaned2.csv", index= False)

In [254]:
df['budget'].value_counts()

10000000.0     55
5000000.0      52
20000000.0     49
1000000.0      45
4000000.0      42
               ..
2322000.0       1
47900000.0      1
4300000.0       1
7700000.0       1
350000000.0     1
Name: budget, Length: 509, dtype: int64

In [255]:
df

,imdb_id,title,genres,popularity,release_date,budget,revenue,vote_average,ROI
0,tt1935859,Miss Peregrine's Home for Peculiar Children,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",306.289,2016-09-28,110000000.0,296482446.0,6.7,2.695295
2,tt4094724,The Purge: Election Year,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",131.741,2016-06-29,10000000.0,118587880.0,6.4,11.858788
3,tt5311514,Your Name.,"[{'id': 10749, 'name': 'Romance'}, {'id': 16, ...",130.125,2016-08-26,370000000.0,357986087.0,8.6,0.967530
4,tt3470600,Sing,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",96.344,2016-11-23,75000000.0,634151679.0,7.0,8.455356
5,tt2119532,Hacksaw Ridge,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",104.716,2016-10-07,40000000.0,175302354.0,8.1,4.382559
...,...,...,...,...,...,...,...,...,...
45857,tt9199374,Uriyadi,[],0.840,2020-01-17,3997866.0,11327287.0,5.0,2.833333
47199,tt10157810,Fools in the Mountains,"[{'id': 35, 'name': 'Comedy'}]",0.791,2020-02-21,21000000.0,1020756.0,5.5,0.048607
47509,tt9861288,Boutchou,"[{'id': 35, 'name': 'Comedy'}]",0.846,2020-09-23,4200000.0,1227428.0,5.5,0.292245
47911,NaN,ALL COPS AIN'T BAD,"[{'id': 35, 'name': 'Comedy'}]",1.230,2020-01-01,100000.0,350000.0,10.0,3.500000


In [263]:
df.drop(list(df.budget.sort_values(ascending=False).head(50).index()), inplace =True)

TypeError: 'Int64Index' object is not callable

In [257]:
df.loc[42019]

imdb_id                                                tt12491064
title                                                    Intruder
genres          [{'id': 9648, 'name': 'Mystery'}, {'id': 53, '...
popularity                                                  4.036
release_date                                           2020-06-04
budget                                                    6.5e+09
revenue                                               3.98445e+06
vote_average                                                  7.5
ROI                                                   0.000612992
Name: 42019, dtype: object